In [3]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata('MNIST original')
Xdata,ydata = mnist["data"],mnist["target"]
#len(X)

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelBinarizer

def split_train_test(X, y, test_ratio=0.2):
    shuffled_indices = np.random.permutation(len(X))
    test_set_size = int(len(X) * test_ratio)
    test_indices = shuffled_indices[:test_set_size]
    train_indices = shuffled_indices[test_set_size:]
    return X[train_indices], X[test_indices], y[train_indices], y[test_indices]

def scale1(X):
    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    X = X.astype(int)
    return X

def scale2(Y):
    #mean = np.mean(Y)
    #Y = Y - mean
    #Y = Y/10
    Y = Y.astype(int)
    Y.reshape(1,-1)
    return Y

X_train, X_test, y_train, y_test = split_train_test(Xdata,ydata, 0.2)

encoder = LabelBinarizer(sparse_output = True)
#y_train = encoder.fit_transform(y_train)
#y_test = encoder.fit_transform(y_test)

X_train = scale1(X_train)
X_test = scale1(X_test)
y_train = scale2(y_train)
y_test = scale2(y_test)

#print(len(X_train), "train +", len(X_test), "test")

/home/arav/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [16]:
import matplotlib
import matplotlib.pyplot as plt
some_digit = y_train[803]
#some_digit_image some_digit.reshape(28,28)
#plt.imshow(some_digit_image, cmap = matplotlib.cm.binary, interpolation = "nearest")
#plt.axis("off")
#plt.show()
some_digit

8

In [5]:
feature1_columns = tf.contrib.learn.infer_real_valued_columns_from_input(X_train)
#dnn_clf = tf.contrib.learn.DNNClassifier(hidden_units = [300, 100], n_classes = 10, feature_columns = feature1_columns)
#dnn_clf.fit(x = X_train, y = y_train, batch_size = 50, steps = 40000)

In [7]:
from sklearn.metrics import accuracy_score
#y_pred = list(dnn_clf.predict(X_test))
#accuracy_score(y_test,y_pred)

#dnn_clf.evaluate(X_test,y_test)

BUILDING A NETWORK FROM BASIC TENSORFLOW API

In [4]:
n_inputs = 28*28 # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

X = tf.placeholder(tf.float32, shape = (None, n_inputs), name = "X")
y = tf.placeholder(tf.int64, shape = (None), name = "y")

In [5]:
def neuron_layer(X, n_neurons, name, activation = None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2/np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev = stddev)
        W = tf.Variable(init, name = "weights")
        b = tf.Variable(tf.zeros([n_neurons]), name = "biases")
        z = tf.matmul(X, W) + b
        if activation == "relu":
            return tf.nn.relu(z)
        else:
            return z

In [6]:
#with tf.name_scope("dnn1"):
    #hidden1 = neuron_layer(X, n_hidden1, "hidden1", activation = "relu")
    #hidden2 = neuron_layer(hidden1, n_hidden2, "hidden2", activation = "relu")
    #logits = neuron_layer(hidden2, n_outputs, "outputs")

In [7]:
from tensorflow.contrib.layers import fully_connected
with tf.name_scope("dnn2"):
    hidden1 = fully_connected(X, n_hidden1, scope = "hidden1")
    hidden2 = fully_connected(hidden1, n_hidden2, scope = "hidden2")
    logits = fully_connected(hidden2, n_outputs, scope = "outputs", activation_fn = None)

In [8]:
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = y, logits = logits)
    loss = tf.reduce_mean(xentropy, name = "loss")

In [9]:
learning_rate = 0.01
with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)

In [10]:
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [11]:
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [22]:
n_epochs = 400
batch_size = 50

In [16]:
def fetch_batch(batch_index, batch_size):
    a_batch = tf.placeholder(tf.float64, shape = [batch_size,1], name = "x")
    b_batch = tf.placeholder(tf.float64, shape = [batch_size,1], name = "x")
    a_batch = X_train[batch_size * batch_index:batch_size * (batch_index + 1)]
    b_batch = y_train[batch_size * batch_index:batch_size * (batch_index + 1)]
    return a_batch, b_batch

In [ ]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(len(X_train)//batch_size):
            X_batch, y_batch = fetch_batch(batch_size,iteration)
            sess.run(training_op, feed_dict = {X:X_batch, y:y_batch})
            acc_train = accuracy.eval(feed_dict = {X:X_batch, y:y_batch})
            acc_test = accuracy.eval(feed_dict = {X:X_test, y:y_test})
            print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)
            save_path = saver.save(sess, "./my_model_final.ckpt")

0 Train accuracy: nan Test accuracy: 0.15214285
0 Train accuracy: 1.0 Test accuracy: 0.15564285
0 Train accuracy: 1.0 Test accuracy: 0.16535714
0 Train accuracy: 0.6666667 Test accuracy: 0.17528571
0 Train accuracy: 0.25 Test accuracy: 0.1805
0 Train accuracy: 0.4 Test accuracy: 0.18435714
0 Train accuracy: 0.5 Test accuracy: 0.18792857
0 Train accuracy: 0.2857143 Test accuracy: 0.19614285
0 Train accuracy: 0.625 Test accuracy: 0.20071429
0 Train accuracy: 0.6666667 Test accuracy: 0.207
0 Train accuracy: 0.4 Test accuracy: 0.21685715
0 Train accuracy: 0.36363637 Test accuracy: 0.2245
0 Train accuracy: 0.41666666 Test accuracy: 0.23435715
0 Train accuracy: 0.23076923 Test accuracy: 0.24442858
0 Train accuracy: 0.35714287 Test accuracy: 0.25357142
0 Train accuracy: 0.2 Test accuracy: 0.2652857
0 Train accuracy: 0.3125 Test accuracy: 0.27585715
0 Train accuracy: 0.29411766 Test accuracy: 0.28678572
0 Train accuracy: 0.3888889 Test accuracy: 0.29485714
0 Train accuracy: 0.31578946 Test acc

0 Train accuracy: 0.76282054 Test accuracy: 0.7567857
0 Train accuracy: 0.7388535 Test accuracy: 0.7575714
0 Train accuracy: 0.74050635 Test accuracy: 0.7577143
0 Train accuracy: 0.7610063 Test accuracy: 0.75707144
0 Train accuracy: 0.75625 Test accuracy: 0.75692856
0 Train accuracy: 0.7515528 Test accuracy: 0.75778574
0 Train accuracy: 0.75308645 Test accuracy: 0.759
0 Train accuracy: 0.74233127 Test accuracy: 0.75935715
0 Train accuracy: 0.75 Test accuracy: 0.7597857
0 Train accuracy: 0.7818182 Test accuracy: 0.75957143
0 Train accuracy: 0.74698794 Test accuracy: 0.75942856
0 Train accuracy: 0.75449103 Test accuracy: 0.7607143
0 Train accuracy: 0.7619048 Test accuracy: 0.76214284
0 Train accuracy: 0.75739646 Test accuracy: 0.7640714
0 Train accuracy: 0.7529412 Test accuracy: 0.7647857
0 Train accuracy: 0.8128655 Test accuracy: 0.76535714
0 Train accuracy: 0.8023256 Test accuracy: 0.76614285
0 Train accuracy: 0.8150289 Test accuracy: 0.7663571
0 Train accuracy: 0.7873563 Test accuracy

0 Train accuracy: 0.8199357 Test accuracy: 0.8189286
0 Train accuracy: 0.82371795 Test accuracy: 0.82014287
0 Train accuracy: 0.8115016 Test accuracy: 0.82014287
0 Train accuracy: 0.8057325 Test accuracy: 0.8207143
0 Train accuracy: 0.8126984 Test accuracy: 0.82114285
0 Train accuracy: 0.8291139 Test accuracy: 0.8207143
0 Train accuracy: 0.82649845 Test accuracy: 0.81964284
0 Train accuracy: 0.8333333 Test accuracy: 0.8190714
0 Train accuracy: 0.8401254 Test accuracy: 0.819
0 Train accuracy: 0.81875 Test accuracy: 0.8198571
0 Train accuracy: 0.81931466 Test accuracy: 0.81935716
0 Train accuracy: 0.8167702 Test accuracy: 0.8191429
0 Train accuracy: 0.8266254 Test accuracy: 0.8202143
0 Train accuracy: 0.8333333 Test accuracy: 0.8214286
0 Train accuracy: 0.84615386 Test accuracy: 0.8222857
0 Train accuracy: 0.85276073 Test accuracy: 0.82278574
0 Train accuracy: 0.8654434 Test accuracy: 0.82357144
0 Train accuracy: 0.8628049 Test accuracy: 0.82364285
0 Train accuracy: 0.8693009 Test accura

0 Train accuracy: 0.8688172 Test accuracy: 0.84435713
0 Train accuracy: 0.86266094 Test accuracy: 0.845
0 Train accuracy: 0.86295503 Test accuracy: 0.8453571
0 Train accuracy: 0.8525641 Test accuracy: 0.845
0 Train accuracy: 0.8443497 Test accuracy: 0.84507143
0 Train accuracy: 0.83404255 Test accuracy: 0.84592855
0 Train accuracy: 0.82165605 Test accuracy: 0.8472857
0 Train accuracy: 0.8114407 Test accuracy: 0.8472143
0 Train accuracy: 0.80972517 Test accuracy: 0.84742856
0 Train accuracy: 0.814346 Test accuracy: 0.84814286
0 Train accuracy: 0.82526314 Test accuracy: 0.84807146
0 Train accuracy: 0.83613443 Test accuracy: 0.8477857
0 Train accuracy: 0.8469602 Test accuracy: 0.84842855
0 Train accuracy: 0.8577406 Test accuracy: 0.8485
0 Train accuracy: 0.8768267 Test accuracy: 0.8495714
0 Train accuracy: 0.87708336 Test accuracy: 0.8497857
0 Train accuracy: 0.8814969 Test accuracy: 0.8495
0 Train accuracy: 0.8858921 Test accuracy: 0.8502143
0 Train accuracy: 0.88612837 Test accuracy: 0.

0 Train accuracy: 0.8723748 Test accuracy: 0.86292857
0 Train accuracy: 0.88709676 Test accuracy: 0.863
0 Train accuracy: 0.88244766 Test accuracy: 0.863
0 Train accuracy: 0.8762058 Test accuracy: 0.86314285
0 Train accuracy: 0.8764045 Test accuracy: 0.86314285
0 Train accuracy: 0.8782051 Test accuracy: 0.86314285
0 Train accuracy: 0.8832 Test accuracy: 0.8633571
0 Train accuracy: 0.88178915 Test accuracy: 0.8635
0 Train accuracy: 0.8740032 Test accuracy: 0.86392856
0 Train accuracy: 0.86464965 Test accuracy: 0.8642143
0 Train accuracy: 0.87440383 Test accuracy: 0.8637143
0 Train accuracy: 0.88095236 Test accuracy: 0.8640714
0 Train accuracy: 0.8748019 Test accuracy: 0.86414284
0 Train accuracy: 0.86550635 Test accuracy: 0.8642143
0 Train accuracy: 0.8672986 Test accuracy: 0.8642857
0 Train accuracy: 0.8675079 Test accuracy: 0.864
0 Train accuracy: 0.87401575 Test accuracy: 0.8640714
0 Train accuracy: 0.8726415 Test accuracy: 0.8640714
0 Train accuracy: 0.8744113 Test accuracy: 0.86414

...

6 Train accuracy: 0.9537246 Test accuracy: 0.94278574
6 Train accuracy: 0.951522 Test accuracy: 0.94278574
6 Train accuracy: 0.9538288 Test accuracy: 0.9427143
6 Train accuracy: 0.9561305 Test accuracy: 0.94278574
6 Train accuracy: 0.9561798 Test accuracy: 0.9427143
6 Train accuracy: 0.95735127 Test accuracy: 0.9424286
6 Train accuracy: 0.9585202 Test accuracy: 0.94264287
6 Train accuracy: 0.95968646 Test accuracy: 0.943
6 Train accuracy: 0.96308726 Test accuracy: 0.94292855
6 Train accuracy: 0.9642458 Test accuracy: 0.943
6 Train accuracy: 0.96428573 Test accuracy: 0.943
6 Train accuracy: 0.9665552 Test accuracy: 0.94285715
6 Train accuracy: 0.9688196 Test accuracy: 0.94264287
6 Train accuracy: 0.96996665 Test accuracy: 0.94285715
6 Train accuracy: 0.9722222 Test accuracy: 0.94292855
6 Train accuracy: 0.9689234 Test accuracy: 0.94264287
6 Train accuracy: 0.96452326 Test accuracy: 0.94264287
6 Train accuracy: 0.9667774 Test accuracy: 0.9427143
6 Train accuracy: 0.96792036 Test accuracy

6 Train accuracy: 0.9778846 Test accuracy: 0.9407857
6 Train accuracy: 0.97598463 Test accuracy: 0.9407143
6 Train accuracy: 0.975048 Test accuracy: 0.9407143
6 Train accuracy: 0.9712368 Test accuracy: 0.9408572
6 Train accuracy: 0.96934867 Test accuracy: 0.9408572
6 Train accuracy: 0.9674641 Test accuracy: 0.94064283
6 Train accuracy: 0.96462715 Test accuracy: 0.9407857
6 Train accuracy: 0.96848136 Test accuracy: 0.9407857
6 Train accuracy: 0.96660304 Test accuracy: 0.9407857
6 Train accuracy: 0.96186846 Test accuracy: 0.9407143
6 Train accuracy: 0.9590476 Test accuracy: 0.9407143
6 Train accuracy: 0.95528066 Test accuracy: 0.94042856
6 Train accuracy: 0.9553232 Test accuracy: 0.9405
6 Train accuracy: 0.9553656 Test accuracy: 0.9405
6 Train accuracy: 0.9582543 Test accuracy: 0.9405714
6 Train accuracy: 0.95829386 Test accuracy: 0.9405714
6 Train accuracy: 0.9564394 Test accuracy: 0.9407857
6 Train accuracy: 0.9545885 Test accuracy: 0.9409286
6 Train accuracy: 0.95652175 Test accuracy:

7 Train accuracy: 0.9375 Test accuracy: 0.94264287
7 Train accuracy: 0.962963 Test accuracy: 0.9425
7 Train accuracy: 0.9390244 Test accuracy: 0.9425
7 Train accuracy: 0.9879518 Test accuracy: 0.9425
7 Train accuracy: 0.96428573 Test accuracy: 0.94278574
7 Train accuracy: 0.9647059 Test accuracy: 0.943
7 Train accuracy: 0.96511626 Test accuracy: 0.943
7 Train accuracy: 0.9885057 Test accuracy: 0.9425
7 Train accuracy: 0.9886364 Test accuracy: 0.94264287
7 Train accuracy: 0.9775281 Test accuracy: 0.9427143
7 Train accuracy: 0.98888886 Test accuracy: 0.94264287
7 Train accuracy: 0.96703297 Test accuracy: 0.94285715
7 Train accuracy: 0.9456522 Test accuracy: 0.9430714
7 Train accuracy: 0.9569892 Test accuracy: 0.94285715
7 Train accuracy: 0.9680851 Test accuracy: 0.94314283
7 Train accuracy: 0.96842104 Test accuracy: 0.9430714
7 Train accuracy: 0.9375 Test accuracy: 0.94335717
7 Train accuracy: 0.92783505 Test accuracy: 0.9427143
7 Train accuracy: 0.9591837 Test accuracy: 0.94214284
7 Tra

7 Train accuracy: 0.9829787 Test accuracy: 0.943
7 Train accuracy: 0.9830508 Test accuracy: 0.94292855
7 Train accuracy: 0.97890294 Test accuracy: 0.943
7 Train accuracy: 0.9747899 Test accuracy: 0.94292855
7 Train accuracy: 0.9707113 Test accuracy: 0.9434286
7 Train accuracy: 0.97083336 Test accuracy: 0.9441429
7 Train accuracy: 0.9792531 Test accuracy: 0.9439286
7 Train accuracy: 0.9793388 Test accuracy: 0.94385713
7 Train accuracy: 0.97530866 Test accuracy: 0.94385713
7 Train accuracy: 0.9590164 Test accuracy: 0.9432857
7 Train accuracy: 0.9591837 Test accuracy: 0.9430714
7 Train accuracy: 0.9593496 Test accuracy: 0.94278574
7 Train accuracy: 0.94331986 Test accuracy: 0.94278574
7 Train accuracy: 0.9516129 Test accuracy: 0.94292855
7 Train accuracy: 0.95983934 Test accuracy: 0.9432143
7 Train accuracy: 0.972 Test accuracy: 0.94264287
7 Train accuracy: 0.9721116 Test accuracy: 0.94335717
7 Train accuracy: 0.98015875 Test accuracy: 0.9432143
7 Train accuracy: 0.97628456 Test accuracy:

7 Train accuracy: 0.9588689 Test accuracy: 0.94335717
7 Train accuracy: 0.96153843 Test accuracy: 0.94357145
7 Train accuracy: 0.96163684 Test accuracy: 0.9439286
7 Train accuracy: 0.95153064 Test accuracy: 0.9434286
7 Train accuracy: 0.956743 Test accuracy: 0.94335717
7 Train accuracy: 0.96192896 Test accuracy: 0.9435
7 Train accuracy: 0.95696205 Test accuracy: 0.9432857
7 Train accuracy: 0.959596 Test accuracy: 0.9432857
7 Train accuracy: 0.9672544 Test accuracy: 0.94335717
7 Train accuracy: 0.96231157 Test accuracy: 0.9434286
7 Train accuracy: 0.9649123 Test accuracy: 0.9435
7 Train accuracy: 0.97 Test accuracy: 0.9435
7 Train accuracy: 0.97007483 Test accuracy: 0.9432143
7 Train accuracy: 0.9726368 Test accuracy: 0.94364285
7 Train accuracy: 0.9801489 Test accuracy: 0.9432857
7 Train accuracy: 0.9851485 Test accuracy: 0.9432857
